In [21]:
import sys
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
from pprint import pprint
import json
from tabulate import tabulate

ROOT = Path().absolute()
DATASETS_PATH = ROOT / "hloc/pipelines/Blender_Synthetic/datasets"
SFM_PATH = ROOT / "hloc/pipelines/Blender_Synthetic/out"

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))


class color:
    PURPLE = "\033[95m"
    CYAN = "\033[96m"
    DARKCYAN = "\033[36m"
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"
    END = "\033[0m"

In [22]:
def is_hfnet_dataset(ds_name):
    return ds_name.startswith("LOC-")

# read stats for datasets
datasets_stats = []
for p in DATASETS_PATH.rglob("stats.json"):
    ds_name = p.parent.name
    #print(f"  {ds_name}")
    with open(p, "r", encoding="utf8") as f:
        data = json.load(f)
    datasets_stats.append((data, ds_name))

print(f"found {len(datasets_stats)} dataset stats")
# pprint(datasets_stats)

hfnet_prefix = "LOC-"
# read stats for sfm results
sfm_stats = []
sfm_loc_stats = []
for p in SFM_PATH.rglob("stats.json"):
    ds_name = p.parent.parent.parent.name
    sfm_method = p.parent.name.removeprefix(ds_name).removeprefix("-")
    #print(f"  {ds_name} - {sfm_method}")
    with open(p, "r", encoding="utf8") as f:
        data = json.load(f)
    if is_hfnet_dataset(ds_name):
        sfm_loc_stats.append((data, ds_name, sfm_method))
    else:
        sfm_stats.append((data, ds_name, sfm_method))

print(f"found {len(sfm_stats)} sfm stats")
print(f"found {len(sfm_loc_stats)} sfm HF-Net stats")
# pprint(sfm_stats)

found 12 dataset stats
found 29 sfm stats
found 25 sfm HF-Net stats


In [23]:
PLACEHOLDER = "-"

# analyse ds stats
table = []
for data, ds_name in datasets_stats:
    c2c_stats = [x for x in data["stats"] if x["full_name"] == "Average Camera to Camera Centroid Distance"]
    c2c_stats = c2c_stats[0] if len(c2c_stats) > 0 else None
    baseline_stats = [x for x in data["stats"] if x["full_name"] == "Min and Max View Baseline"]
    baseline_stats = baseline_stats[0] if len(baseline_stats) > 0 else None
    table.append(
        [
            f'{data["scene_name"]} ({data["env_background"]})',
            data["num_cameras"],
            PLACEHOLDER if not c2c_stats else f'{c2c_stats["value"]:.2f}',
            PLACEHOLDER if not baseline_stats else f'{baseline_stats["min"]:.2f}',
            PLACEHOLDER if not baseline_stats else f'{baseline_stats["max"]:.2f}',
        ]
    )

print(
    tabulate(
        table,
        [
            "name",
            "cams",
            "avg. c2c centroid dist (m)",
            "min view dist (m)",
            "max view dist (m)",
        ],
        tablefmt="plain",
        showindex="always",
    )
)

    name                              cams    avg. c2c centroid dist (m)    min view dist (m)    max view dist (m)
 0  Bartholomew (evening_field_8k)      10                          4.39                 2.43                14.42
 1  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 2  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 3  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 4  Framlingham (evening_field_8k)      10                         11.07                 4.22                39.68
 5  Bartholomew (evening_field_8k)      10                          4.39                 2.43                14.42
 6  Chateu (evening_field_8k)           15                         11.26                 2.71                32.2
 7  Chateu (evening_field_8k)           15                         11.26            

In [24]:
# analyse ds stats
def get_analysis_table(sfm_stats):
    table = []
    for data, ds_name, sfm_method in sfm_stats:
        if "result" in data and data["result"] == "error":
            table.append(
                [
                    ds_name,
                    f"{sfm_method} *",
                    PLACEHOLDER,
                    PLACEHOLDER,
                    PLACEHOLDER,
                    PLACEHOLDER,
                    PLACEHOLDER,
                    PLACEHOLDER,
                    PLACEHOLDER,
                ]
            )
            continue
        t_errs = np.array([i["t_err_m"] for i in data["analysed_poses"]])
        r_errs = np.array([i["r_err_deg"] for i in data["analysed_poses"]])
        re_errs = np.array([i["reprojection_err"] for i in data["analysed_poses"]])
        t_errs_med = np.median(t_errs)
        r_errs_med = np.median(r_errs)
        loc = [i for i in data["analysed_poses"] if "MAIN" in i["name"]][0]
        loc_t_err = loc["t_err_m"]
        loc_r_err = loc["r_err_deg"]
        re_errs_med = np.median(re_errs)
        loc_abs_ground_distance = loc["ground_distance_m"]

        # ratios
        t_errs_size = t_errs.size
        r_errs_size = r_errs.size
        t_ratio = t_errs[t_errs < 1].size / t_errs_size
        r_ratio = r_errs[r_errs < 5].size / r_errs_size

        table.append(
            [
                ds_name,
                sfm_method,
                f"{re_errs_med:.2f}",
                f"{t_errs_med:.2f}",
                f"{r_errs_med:.2f}",
                f"{loc_t_err:.2f}",
                f"{loc_r_err:.2f}",
                f"{loc_abs_ground_distance:.2f}",
                # f"{t_ratio:.2f}",
                # f"{r_ratio:.2f}",
            ]
        )

    columns = [
        "dataset",
        "feature pipeline",
        "med. reprojection error (px)",  # median, because main cam is outlier
        "med. t_err (m)",
        "med. r_err (°)",
        "localized camera t_err (m)",
        "localized camera r_err (°)",
        "distance to ground (m)",
        # "t_err (x < 1px)",
        # "r_err (x < 5°)",
    ]

    return table, columns


sfm_stats_table, sfm_stats_table_cols = get_analysis_table(sfm_stats)

print(
    tabulate(
        sfm_stats_table,
        sfm_stats_table_cols,
        tablefmt="plain",
        showindex="always",
    ),
    "\n\n* this method failed to localize the camera",
)

    dataset                       feature pipeline        med. reprojection error (px)    med. t_err (m)    med. r_err (°)    localized camera t_err (m)    localized camera r_err (°)    distance to ground (m)
 0  Bartholomew+evening_field_8k  disk+lightglue          0.87                            0.03              0.21              1.29                          2.39                          1.11
 1  Bartholomew+evening_field_8k  loftr                   0.60                            0.01              0.05              1.58                          2.17                          0.88
 2  Bartholomew+evening_field_8k  superpoint+lightglue    1.12                            0.02              0.21              4.97                          1.62                          0.97
 3  Bartholomew+evening_field_8k  superpoint+superglue    1.11                            0.02              0.18              4.12                          1.64                          1.00
 4  Chateu-img1+evening_fie

In [25]:
# bold text below certain threshold
def bold_print_threshold(value: float, threshold: float = None):
    f_value = f"{value:.2f}"
    if not threshold or value < threshold:
        return f"{color.BOLD}{f_value}{color.END}"
    return f_value


# aggregate hfnet and non-hfnet stats for comparison
sfm_hfnet_stats_table = []
sfm_hfnet_stats_table_cols = [
    "DS Name",
    "feature pipeline",
    "localized camera t_err (m)",
    "localized camera r_err (°)",
    "distance to ground (m)",
]

for i, stat in enumerate(sfm_stats):
    stat_ds_data = stat[0]
    stat_ds_name = stat[1]
    stat_ds_method = stat[2]

    matching_hfnet_stat = None
    for j, hfnet_stat in enumerate(sfm_loc_stats):
        hfnet_ds_name = hfnet_stat[1]
        hfnet_ds_method = hfnet_stat[2]

        if stat_ds_name in hfnet_ds_name and stat_ds_method == hfnet_ds_method:
            matching_hfnet_stat = hfnet_stat
            break

    # if matching_hfnet_stat:
    #    print(f"found matching HF-Net stats for {stat_ds_name} / {matching_hfnet_stat[1]} for method {stat_ds_method} / {matching_hfnet_stat[2]}")
    # else:
    #    print(f"no matching HF-Net stats for {stat_ds_name} for method {stat_ds_method}")

    A_stats = []
    B_stats = []

    # A stats for SFM
    if "result" in stat_ds_data and stat_ds_data["result"] == "error":
        A_stats = [PLACEHOLDER, PLACEHOLDER, PLACEHOLDER]
    else:
        loc = [i for i in stat_ds_data["analysed_poses"] if "MAIN" in i["name"]][0]
        loc_t_err = loc["t_err_m"]
        loc_r_err = loc["r_err_deg"]
        loc_abs_ground_distance = loc["ground_distance_m"]
        A_stats = [
            f"{loc_t_err:.2f}",
            f"{loc_r_err:.2f}",
            f"{loc_abs_ground_distance:.2f}",
        ]

    # B stats for HF-Net
    if (
        matching_hfnet_stat is None
        or "result" in matching_hfnet_stat[0]
        and matching_hfnet_stat[0]["result"] == "error"
    ):
        B_stats = [PLACEHOLDER, PLACEHOLDER, PLACEHOLDER]
    else:
        hfnet_data = matching_hfnet_stat[0]
        hfnet_loc = [i for i in hfnet_data["analysed_poses"] if "MAIN" in i["name"]][0]
        hfnet_loc_t_err = hfnet_loc["t_err_m"]
        hfnet_loc_r_err = hfnet_loc["r_err_deg"]
        hfnet_loc_abs_ground_distance = hfnet_loc["ground_distance_m"]
        B_stats += [
            f"{hfnet_loc_t_err:.2f}",
            f"{hfnet_loc_r_err:.2f}",
            f"{hfnet_loc_abs_ground_distance:.2f}",
        ]

    # combine row general info with stats from A and B
    row = [stat_ds_name, stat_ds_method]  # partial row
    for a, b in zip(A_stats, B_stats):
        row.append(f"{a} / {b}")

    sfm_hfnet_stats_table.append(row)

print(
    tabulate(
        sfm_hfnet_stats_table,
        sfm_hfnet_stats_table_cols,
        tablefmt="plain",
        showindex="always",
    )
)

    DS Name                       feature pipeline      localized camera t_err (m)    localized camera r_err (°)    distance to ground (m)
 0  Bartholomew+evening_field_8k  disk+lightglue        1.29 / 2.13                   2.39 / 9.96                   1.11 / -0.33
 1  Bartholomew+evening_field_8k  loftr                 1.58 / 27.66                  2.17 / 143.92                 0.88 / 22.22
 2  Bartholomew+evening_field_8k  superpoint+lightglue  4.97 / 3.26                   1.62 / 12.55                  0.97 / -1.01
 3  Bartholomew+evening_field_8k  superpoint+superglue  4.12 / 3.34                   1.64 / 13.09                  1.00 / -1.13
 4  Chateu-img1+evening_field_8k  disk+lightglue        - / 29.91                     - / 105.54                    - / 2.36
 5  Chateu-img1+evening_field_8k  loftr                 - / 5846.81                   - / 191.12                    - / -5828.32
 6  Chateu-img1+evening_field_8k  r2d2+nn_ratio         - / -                         - / -